# **ICT3464 - Ruteo de Vehículos**
### Taller de preparación Trabajo 1: Resolviendo el TSP con heurísticas

Profesor: Homero Larraín Izquierdo (hlarraii@uc.cl)

Ayudantes: Diego San Martín (dsm@uc.cl), Benjamín Rojas (bgrojas@uc.cl)

## Parte 0: Algunas funciones útiles

In [2]:
def calcular_costos(Ruta, Distancias, C_V, C_F):

    # Ambos costos parten en 0
    costos_variables = 0
    costos_fijos = 0

    # Calculamos los costos varibales
        # Agregamos el costo de cada arco
    for it_nodo in range(len(Ruta)-1):
        n_s = Ruta[it_nodo] # Nodo de salida
        n_e = Ruta[it_nodo + 1] # Nodo de entrada
        costos_variables += C_V * Distancias[n_s][n_e]/1000

    # Calculamos los costos fijos
    costos_fijos += C_F

    # Sumamos ambos costos
    costos = costos_variables + costos_fijos

    return costos, costos_variables, costos_fijos

#### Función para verificar que una solución es factible

## Parte I-1: Matriz de costos, importar datos del problema y visualización

#### Función para generar una matriz de costos a partir de la matriz de distancias, costos variables y costos fijos

In [5]:
# Tamaño de la instancia
n = 10

In [6]:
file = "Instancias TSP\\Talca-n" + str(n) + ".pkl"

with open(file, 'rb') as file:
    coordenadas = pickle.load(file) # Latitud, longidut de cada nodo
    caminos = pickle.load(file) # Caminos mínimos entre cada par de nodos. Lo usaremos más adelante para la visualización de las rutas en osmnx/folium.
    distancias = pickle.load(file) # Distancias entre cada par de nodos (m)
    demandas = pickle.load(file) # Demandas de los clientes. Para el nodo depot la demanda es 0. (u)
    capacidad = pickle.load(file) # Capacidad de los vehículos (u)
    c_v = pickle.load(file) # Costos variables ($/km)
    c_f = pickle.load(file) # Costos fijos ($/veh.)

# Creamos la matriz de costos
costos = generar_matriz_de_costos(distancias, c_v, c_f)

In [7]:
# G es el grafo de Talca
place_name = 'Talca, Chile'

G = ox.graph_from_place(place_name, network_type='drive')

# Generamos un mapa en folium centrado en el Plaza de Armas de Talca
m = folium.Map(location=[-35.426527777778, -71.666055555556], zoom_start=10, tiles='OpenStreetMap')

# Inicializamos un contador de nodos
i = 0

# Agregamos cada nodo en el mapa
for coord in coordenadas:
    icon = folium.Icon(icon="fa-solid fa-user",
                       prefix='fa',
                       color='red')
    folium.Marker(location=[coord[0], coord[1]],
                  popup=folium.Popup(f'Nodo {i}', max_width=75),
                  icon=icon).add_to(m)
    i += 1

display(m)

## Parte I-2: Heurística constructiva

In [8]:
def construccion_aleatoria(Costos):

    # Inicializamos las rutas como una lista vacía
    Ruta = []

    # Costos es una matriz cuadrada de n x n (siendo n el número de clientes)
    # Calculamos cuantos clientes tiene la instancia
    n = len(Costos)


    # Generamos una lista con todos los clientes
    lista_clientes = []
    for cliente in range(0, n):
        lista_clientes.append(cliente)

    # Fijamos una semilla para reproducibilidad
    np.random.seed(654321)

    # Mientras no se cumpla el criterio de salida, incorporamos de manera aleatoria clientes a nuestra ruta
    while True:

        # Generamos un índice aleatorio de la lista
        indice_aleatorio = np.random.randint(0, len(lista_clientes))

        # Incorporamos el cliente asociado a ese índice
        Ruta.append(lista_clientes[indice_aleatorio])

        # Eliminamos el cliente de 'lista_clientes'
        lista_clientes.remove(lista_clientes[indice_aleatorio])

        # Criterio de salida: se han incorporado todos los clientes en la ruta
        if len(lista_clientes) == 0:
            break

    # La ruta no necesariamente comenzará en el nodo indexado por 0. Reorganizamos la ruta para que parta desde 0
     # Encontrar el índice del número 0 en la lista
    indice_cero = Ruta.index(0)

     # Partir la lista en dos partes y luego unirlas en el orden deseado
    Ruta = Ruta[indice_cero:] + Ruta[:indice_cero]

    # Agregamos el arco de vuelta al nodo indexado por 0
    Ruta.append(0)

    return Ruta

In [9]:
sol_aleatoria = construccion_aleatoria(costos)
print(sol_aleatoria, es_factible(sol_aleatoria, n), calcular_costos(sol_aleatoria, distancias, c_v, c_f))

[0, 1, 8, 5, 4, 6, 3, 9, 2, 7, 0] La ruta es factible. (18085.0, 17085.0, 1000)


In [10]:
def clarke_and_wright(Costos):

    # Inicializamos las rutas como una lista vacía
    Ruta = []

    # Costos es una matriz cuadrada de n x n (siendo n el número de clientes)
    # Calculamos cuantos clientes tiene la instancia
    n = len(Costos)

    # Calculamos los ahorros con la ecuación
        # s_ij = c_0i + c_i0 + c_0j + c_j0 - (c_0i + c_ij + c_j0)
        # s_ij = c_i0 + c_j0 - cij

    # Nota. En esta implementación se asume por defecto que el nodo que viene indexado por 0 es el depot ficticio.
    # Se puede aleatorizar y partir con un nodo depot ficticio distinto del indexado por 0.

    # s: lista de ahorros (savings)
    s = dict()
    for i in range(n):
        for j in range(n):
            if i!=j and i!=0 and j!=0:
                arc = (i, j)
                savings = round(Costos[i][0] + Costos[0][j] - Costos[i][j], 1)
                s[arc] = savings


    # Ordenamos en sentido decreciente (nos interesan los que tienen mayor ahorro)
    s = dict(sorted(s.items(), key=lambda item: item[1], reverse = True))

    # Nota. Clarke y Wright se basa en la desigualdad triangular, caso que no sucede necesariamente en la realidad. Eventualmente vamos a tener ahorros negativos.
    # De todas formas van a quedar al final del ranking. En particular, no los borramos para evitar cualquier tipo de infactibilidad.

    # Generamos una lista con identificadores del árbol al que pertenece cada nodo, para evitar cerrar subtours de manera eficiente
    # (Clase de heurísticas del TSP)
    id_arbol = []
    for i in range(n):
        id_arbol.append(i)

    # Generamos un grafo para llevar seguimiento de los arcos conectados entre sí y luego poder recrear la solución
    G = nx.DiGraph()

    # Mientras no se cumpla el criterio de salida
    while True:
        # Tomamos como arco seleccionado al primero de nuestra lista de savings
        arco_seleccionado = next(iter(s.items()))[0] # llave (i, j) del primer elemento de s
        n_s = arco_seleccionado[0] # cola del arco
        n_e = arco_seleccionado[1] # cabeza del arco

        # Evaluamos si el arco nos genera un subtour (si une dos árboles iguales, entonces lo genera)
        if id_arbol[n_s] != id_arbol[n_e]:

            # Agregamos el arco a nuestro grafo G
            G.add_edge(n_s, n_e)

            # Buscamos los nodos/componentes conectados entre sí, aprovechando la función nx.weakly_connected_components
            componentes = list(nx.weakly_connected_components(G))

            # Actualizamos id_arbol:
             # Buscamos en cada arbol
            for lista_componentes in componentes:
                # Buscamos el mínimo de cada lista de componentes conectados
                id_arbol_min = np.inf
                for nodo in lista_componentes:
                    if id_arbol[nodo] < id_arbol_min:
                        id_arbol_min = id_arbol[nodo]

                # Igualamos todos los id_arbol al mínimo
                for nodo in lista_componentes:
                    id_arbol[nodo] = id_arbol_min

            # Estamos resolviendo un TSP dirigido
            # por lo tanto tenemos que eliminar todos los arcos que salgan desde la cola del arco recien incorporado
            # y todos los arcos que entren a la cabeza del arco recien incorporado
            s = {(i, j): value for (i, j), value in s.items() if i != n_s}
            s = {(i, j): value for (i, j), value in s.items() if j != n_e}

        # Si el arco nos genera un subtour, lo eliminamos de la lista de ahorros y continuamos
        else:
            del s[(n_s, n_e)]

        # Criterio de salida: la lista de savings no tiene elementos
        if len(s) == 0:
            break


    # Transformamos la lista de arcos en una ruta:
     # Para esto, primero buscamos los nodos s y t de nuestro s-t camino. Estos corresponderán a los los dos nodos de grado 1 de nuestros grafo
    grados = G.degree()
    nodos_grado_1 = [nodo for nodo, grado in grados if grado == 1]

     # Buscamos cuál es s y cuál es t
    edges = list(G.edges())
    for e in edges:
        for n in nodos_grado_1:
            if n == e[0]:
                n_s = e[0]
            if n == e[1]:
                n_t = e[1]

     # Generamos nuestro s-t camino
    nodo_actual = n_s
    Ruta.append(nodo_actual)

    while True:
        for e in edges:
            if e[0] == nodo_actual:
                nodo_actual = e[1]
                Ruta.append(nodo_actual)
                edges.remove(e)
                break

        if nodo_actual == n_t:
            break

     # Agregamos el nodo 0 al inicio y final del s-t camino, para transformarlo en la ruta correspondiente
    Ruta.insert(0, 0)
    Ruta.append(0)

    return Ruta

In [11]:
sol_cw = clarke_and_wright(costos)
print(sol_cw, es_factible(sol_cw, n), calcular_costos(sol_cw, distancias, c_v, c_f))

[0, 6, 4, 7, 3, 9, 8, 1, 2, 5, 0] La ruta es factible. (9792.7, 8792.7, 1000)


#### 2-opt enfoque best improvement

In [12]:
def dos_opt(Ruta, Distancias, C_V, C_F):

    # Calculamos el costo actual
    Costo_Actual = calcular_costos(Ruta, Distancias, C_V, C_F)[0]

    # Genero una lista para todas las rutas mejoradas y los valores de sus F.O.
    Rutas_Improvement = []
    Costos_Improvement = []

    # Aplico 2-opt para cada par posible, recordando que es un problema asimétrico.
    for i in range(1, len(Ruta) - 2):
        for j in range(1, len(Ruta)- 2):
                New_Ruta = Ruta[:i] + list(reversed(Ruta[i:j+1])) + Ruta[j+1:]
                Costo_New_Ruta = calcular_costos(New_Ruta, Distancias, C_V, C_F)[0]
                if Costo_New_Ruta < Costo_Actual:
                    Rutas_Improvement.append(New_Ruta)
                    Costos_Improvement.append(Costo_New_Ruta)

    # Si hubieron mejoras, seleccionamos la mejor de estas
    if len(Costos_Improvement) != 0:
        # Encontrar el mínimo
        min_costo = min(Costos_Improvement)

        # Obtener el índice correspondiente al mínimo
        indice_minimo = Costos_Improvement.index(min_costo)

        # Actualizamos
        Ruta = Rutas_Improvement[indice_minimo].copy()
        Costo_Actual = min_costo

    return Ruta


#### Relocate enfoque best improvement

In [14]:
def relocate(Ruta, Distancias, C_V, C_F):

    # Calculamos el costo actual
    Costo_Actual = calcular_costos(Ruta, Distancias, C_V, C_F)[0]

    # Genero una lista para todas las rutas mejoradas y los valores de sus F.O.
    Rutas_Improvement = []
    Costos_Improvement = []

    # Aplico relocate para cada par posible, recordando que es un problema asimétrico.
    for i in range(1, len(Ruta) - 2):
        for j in range(1, len(Ruta)- 1):
                if abs(i-j) >= 1:
                    New_Ruta = Ruta.copy() # Copiamos la ruta
                    New_Ruta.remove(Ruta[i]) # Eliminamos el nodo i
                    New_Ruta.insert(j, Ruta[i]) # Insertamos el nodo i luego del indice j

                    Costo_New_Ruta = calcular_costos(New_Ruta, Distancias, C_V, C_F)[0]
                    if Costo_New_Ruta < Costo_Actual:
                        Rutas_Improvement.append(New_Ruta)
                        Costos_Improvement.append(Costo_New_Ruta)

    # Si hubieron mejoras, seleccionamos la mejor de estas
    if len(Costos_Improvement) != 0:

        # Encontrar el mínimo
        min_costo = min(Costos_Improvement)

        # Obtener el índice correspondiente al mínimo
        indice_minimo = Costos_Improvement.index(min_costo)

        # Actualizamos
        Ruta = Rutas_Improvement[indice_minimo].copy()
        Costo_Actual = min_costo

        # print("Mejoró, ahora el costo actual es: ", Costo_Actual)

    return Ruta


#### Óptimalidad local

In [17]:
sol_aleatoria_opt_local_relocate = optimalidad_local(sol_aleatoria, costos, distancias, c_v, c_f, relocate)
print(sol_aleatoria_opt_local_relocate, es_factible(sol_aleatoria_opt_local_relocate, n), calcular_costos(sol_aleatoria_opt_local_relocate, distancias, c_v, c_f))

[0, 5, 2, 1, 8, 9, 3, 7, 4, 6, 0] La ruta es factible. (9958.0, 8958.0, 1000)


## Parte I-4: Visualización del mapa interactivo

In [19]:
print(r, calcular_costos(r, distancias, c_v, c_f)[0])

[0, 5, 2, 1, 8, 9, 3, 7, 4, 6, 0] 9958.0


In [20]:
display(m)

In [21]:
def busqueda_local_calibrada(Ruta, Costos, Distancias, C_V, C_F):

    Costo_Actual = calcular_costos(Ruta, Distancias, C_V, C_F)[0]

    # Definimos probabilidad y opciones
    opciones = [0, 1] # Tomaremos 0 para 2opt, 1 para relocate
    probabilidad = 0.5 # Probabilidad de usar relocate
    pesos = [1 - probabilidad, probabilidad]

    # Ciclo de optimalidad local:
    improvement = True
    while improvement:
        improvement = False

        # Generamos un ensayo Bernoulli
        resultado = np.random.choice(opciones, size=1, p=pesos)

        # Si es 0, aplicamos 2opt
        if resultado == 0:
            Ruta_Candidata = dos_opt(Ruta, Distancias, C_V, C_F)
        # Si es 1, aplicamos relocate
        else:
            Ruta_Candidata = relocate(Ruta, Distancias, C_V, C_F)

        # Verificamos si ha mejorado
        Costo_Ruta_Candidata = calcular_costos(Ruta_Candidata, Distancias, C_V, C_F)[0]

        # Si aplicar el operador genera una mejora, entonces actualizamos el costo actual, la ruta y volvemos a dejar como True el indicador 'improvement'
        if Costo_Ruta_Candidata < Costo_Actual:
            print("Se generó una mejora de: ", (Costo_Actual - Costo_Ruta_Candidata))
            Costo_Actual = Costo_Ruta_Candidata
            Ruta = Ruta_Candidata.copy()
            improvement = True

        ### IMPORTANTE: si no mejoró utilizando el último operador, no significa que sea óptimo local respecto al otro operador

        if improvement == False:
            # Si el último operador utilizado fue 2opt, entonces aplicamos relocate para ver si es óptimo local respecto a ambos
            if resultado == 0:
                Ruta_Candidata = relocate(Ruta, Distancias, C_V, C_F)
            # Lo contrario si el último operador utilizado fue relocate
            else:
                Ruta_Candidata = dos_opt(Ruta, Distancias, C_V, C_F)

        # Verificamos si ha mejorado
        Costo_Ruta_Candidata = calcular_costos(Ruta_Candidata, Distancias, C_V, C_F)[0]

        # Si aplicar el operador genera una mejora, entonces actualizamos el costo actual, la ruta y volvemos a dejar como True el indicador 'improvement'
        if Costo_Ruta_Candidata < Costo_Actual:
            print("No era óptimo local respecto a ambos. Se generó una mejora de: ", (Costo_Actual - Costo_Ruta_Candidata))
            Costo_Actual = Costo_Ruta_Candidata
            Ruta = Ruta_Candidata.copy()
            improvement = True


    return Ruta

## Parte II-2: Solución del set de instancias

#### Puede ser útil: leer todos los archivos desde la carpeta 'Instancias' directamente